In [126]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

기사 개별 url 가져오기

In [178]:
url = 'https://search.daum.net/search'

search_keywords = ['도시형 스마트팜', '도심형 스마트팜', '수직형 스마트팜']

params = {
    'w': 'news',
    'nil_search': 'btn',
    'DA': 'PGD',
    'enc': 'utf8',
    'cluster': 'y',
    'cluster_page': '1',
    'q': '수직형 스마트팜',  # 수직형 스마트팜, 도심형 스마트팜, 도시형 스마트팜
    'show_dns': '1',
    'sd': '20200101000000',  # 뉴스 검색 시작 날짜
    'ed': '20230930235959',  # 뉴스 검색 마지막 날짜 20230930235959
    'period': 'u',
    'p': 1,  # 페이지 1부터 같은 페이지가 나올 때까지 while True로 구현 += 1
    'sort': 'old',
}

# 기사의 개별 url 가져오는 함수
def get_urls(soup):
    urls = []
    for item in soup.select('.item-title'):
        urls.append(item.select_one('a').attrs['href'])

    return urls

# 한 화면에 기사 20개가 담긴 페이지에서, 원하는 기간 동안, 기사 개별 url 가져오기 (페이지 넘김 기능)

result = []
previous_urls = None
while True:
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = get_urls(soup)  
    # current_page_content = str(soup)

    if previous_urls == urls:
        break

    # time.sleep(1)
    print(f"Page {params['p']} URLs:", urls)

    params['p'] += 1
    previous_urls = urls
    result.extend(urls)
print(len(result))

Page 1 URLs: []
0


다음 뉴스 본문 가져오기

In [174]:
def get_articles_to_df(result):
    articles = []
    for url in result:
        article = {}
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one('#mArticle > div.head_view > h3').text
        date = ''.join([data.zfill(2) for data in re.findall(r'\d+', soup.select_one('.info_view').text)[:3]])
        # date = soup.select_one('#mArticle > div.head_view > div.info_view > span:nth-child(2)').text
        content = re.sub('\n|\t|\r','', soup.select_one('.article_view').text.strip())

        article = {
            'title' : title,
            'date' : date,
            'content' : content
        }
        articles.append(article)
    df = pd.DataFrame(articles)
    df.to_csv(f"articles_{params['q']}_{params['sd'][:8]}_.csv")
    return df
    
df = get_articles_to_df(result)

In [175]:
df

,title,date,content
0,미래를 여는 열쇠 사람과 환경 중심의 농업 농촌,20200221,국민께서 확실한 변화를 체감할 수 있도록 ‘사람과 환경 중심의 농정개혁’의 틀을 정...
1,"진주乙 강민국, 젊은 심장 뛰는 진주 선언",20200407,[김동수 기자(=진주)(kdsu21@naver.com)] 진주 '...
2,"안동시, 대도약 10대 프로젝트 추진..정책간담회서 논의",20200423,[안동=뉴시스] 김진호 기자 = 안동문화예술의전당 국제회의장에서 예산정책 간담회가 ...
3,"안동시, '안동형 일자리'로 '포스트 코로나19' 올(All) 극복",20200507,경북 안동시가 '안동형 일자리'로 '포스트 코로나19'를 올(All) 극복한다. 3...
4,"[인터뷰]김상범 서울교통공사 사장""시민 신뢰 얻을 수 있는 최고 수준 안전 확보 최...",20200620,[아시아경제 박종일 기자] 김상범 서울교통공사 사장이 20일로 취임 2개월20일을 ...
...,...,...,...
271,방치됐던 명동 건물···최첨단 '딸기농장' 들어섰다,20230908,8일 서울 중구 명동에 문을 연 스마트팜 기업 넥스트온의 ‘매드베리팜하우스’를 찾은...
272,푸르메 재단·병원·농장 이어 '마을'…장애인을 위한 꿈은 계속 된다,20230911,"[이명선 기자(overview@pressian.com)]""1%의 우연이 필연이 되는..."
273,텃밭 전문가 키우고 맞춤형 원예교육 … ‘도시농업 메카’ 일궈 [지방기획],20230913,부산은 국내 대도시 가운데 엽채류·과채류·화훼류를 재배하는 근교농업과 시설원예가 발...
274,글로컬 상생 위해 순천시장-서울시교육감-순천대총장 뭉쳤다,20230914,[문경화 기자(=순천)(988810@hanmail.net)]노관규 순천시장은 조희연...


In [176]:
df.to_csv(f"articles_{params['q']}_{params['sd'][:8]}_.csv")

---
json dump 필요시

In [14]:
# list -> json 파일로 내보내기
import json

file_path = f"수직_농장_{len(result)}.json"

# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(result, json_file)

print(f"list -> json 파일로 내보내기 완료 {file_path}")

list -> json 파일로 내보내기 완료 수직_농장_800.json


In [57]:
# json 파일 -> list 로 읽기
with open(file_path, "r") as json_file:
    result= json.load(json_file)
print(result)

['http://v.daum.net/v/20200102134531371', 'http://v.daum.net/v/20200103144228911', 'http://v.daum.net/v/20200104140154605', 'http://v.daum.net/v/20200104140208609', 'http://v.daum.net/v/20200105150102692', 'http://v.daum.net/v/20200106105905621', 'http://v.daum.net/v/20200107060020898', 'http://v.daum.net/v/20200107060144002', 'http://v.daum.net/v/20200107073915065', 'http://v.daum.net/v/20200107133159246', 'http://v.daum.net/v/20200108141118834', 'http://v.daum.net/v/20200108172602390', 'http://v.daum.net/v/20200109091153730', 'http://v.daum.net/v/20200109091330814', 'http://v.daum.net/v/20200113164605877', 'http://v.daum.net/v/20200113165038076', 'http://v.daum.net/v/20200113173903748', 'http://v.daum.net/v/20200115152330118', 'http://v.daum.net/v/20200116031210375', 'http://v.daum.net/v/20200116115551378', 'http://v.daum.net/v/20200117102018458', 'http://v.daum.net/v/20200117141235123', 'http://v.daum.net/v/20200120114306705', 'http://v.daum.net/v/20200120122635140', 'http://v.daum.

---
참고
- 리팩토링하면서 삭제된 코드

In [ ]:
# 날짜 형식 바꾸기 함수 생성
def modify_date(row):
    tmp = row.split('.')
    year = tmp[0]
    if len(tmp[1]) == 1:
        month = '0' + tmp[1]
    else:
        month = tmp[1]
    if len(tmp[2]) == 1:
        day = '0' + tmp[2]
    else:
        day = tmp[2]
    date = year + month + day
    return date

df['date'] = df['date'].apply(modify_date)

"""
위 함수 생성해서 apply 적용할 필요 없이 기사 날짜 가져올 때, zfill로 해결 가능함!
''.join([data.zfill(2) for data in re.findall(r'\d+', soup.select_one('.info_view').text)[:3]])
"""